## Notebook for plotting the eigenmodes of the simple isospectral domains computed using $\mathcal{P}_1$ finite elements.

The eigenmodes are drawn for the ground state as well as the 9-th excited state. For the latter analytic solutions exist in the case of $\omega=0$

Experiment data. The eigenmodes are indexed by angular frequency $\omega$ in a dictionary, which itself is indexed by a tuple containing the domain prefix, the index of refraction and the mode number.
**Note**: The data has been collected manually and the plots are not fully automated. To change the eigenvalue to be drawn one should change the indices of the modes to be drawn. The appropriate indices are provided below. If an index is negative one should prepend a minus sign to the eigenmode to invert it.

In [1]:
DataLin = {("ID1_1", 1, 1):{0:[-1,2], 661.53900:[1,2],58891200.00000:[1,-3],132919000.00000:[0,3] },
        ("ID1_1", 2.42, 1):{0:[0,1], 661.53900:[1,2],58891200.00000:[1,-3],132919000.00000:[0] },
        ("ID1_1", 1, 9):{0:[16,17], 661.53900:[16,17],58891200.00000:[16,-17],132919000.00000:[16,-17] },
        ("ID1_1", 2.42, 9):{0:[16,17], 661.53900:[16,-17],58891200.00000:[16,-17],132919000.00000:[16,-17] },
        ("ID2_1", 1, 1):{0:[0,2], 661.53900:[0,-3],58891200.00000:[0,1],132919000.00000:[0,1] },
        ("ID2_1", 2.42, 1):{0:[0,-2], 661.53900:[0,2],58891200.00000:[0,1],132919000.00000:[0,1] }
       };
DataFull = {("ID1_1", 1, 1):{0:[1,2], 661.53900:[0,1],58891200.00000:[1],132919000.00000:[1] },
        ("ID1_1", 2.42, 1):{0:[0,1], 661.53900:[0,1],58891200.00000:[0],132919000.00000:[0] },
        ("ID1_1", 1, 9):{0:[16,-17], 661.53900:[16,-17],58891200.00000:[16,-17],132919000.00000:[16,-17] },
        ("ID1_1", 2.42, 9):{0:[16,-17], 661.53900:[16,17],58891200.00000:[16,-17],132919000.00000:[16,-17] },
        ("ID2_1", 1, 1):{0:[0,2], 661.53900:[0,-2],58891200.00000:[0,1],132919000.00000:[2,3] },
        ("ID2_1", 2.42, 1):{0:[0,-3], 661.53900:[0,-3],58891200.00000:[1,2],132919000.00000:[0,1] }
       };
Folders = { "ID1_1 linear": "../data/ID1/P1_EigVecExperiment",
          "ID1_1 full" : "../data/ID1/P1_Full_EigVecExperiment",
          "ID2_1 linear" : "../data/ID2/P1_EigVecExperiment",
          "ID2_1 full" : "../data/ID2/P1_Full_EigVecExperiment"};

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.tri as tri
from scipy.constants import codata
import scipy as sp
import matplotlib.colors
from mpl_toolkits.axes_grid1 import AxesGrid
import os, shutil, sys
from PlotFunctions import *
c = sp.constants.value('speed of light in vacuum');

Define the path where the data can be found and change into the directory

In [3]:
for param in DataLin.keys():
    folder = ""
    if param[0] == "ID1_1" :
        folder = Folders["ID1_1 linear"];
    else:
        folder = Folders["ID1_1 full"];
    experiment = ""
    if param[1] == 1:
        experiment = "Exp_0";
    else:
        experiment = "Exp_1";
    destination = folder + '/' + experiment
    os.chdir(destination)

In [4]:
cd ../data/P1_Full_EigVecExperiment/Exp_0

/home/shared/FEM_EV_NumExp/Data/CorrectedShift/P1_Full_EigVecExperiment/Exp_0


Define the domain prefix.

In [5]:
domain = "ID1_2"

In [6]:
vertexFilename = "Plot_Vertices_"+domain+".dat"
elementFilename = "Plot_Elements_"+domain+".dat"
ModeFile_low = "Eigenvalues_"+domain+"_omega_661.53900.dat"
ModeFile_med = "Eigenvalues_"+domain+"_omega_58891200.00000.dat"
ModeFile_high = "Eigenvalues_"+domain+"_omega_132919000.00000.dat"

Indices of the eigenvectors which should be drawn

In [7]:
GSIdx_stat = 2;
NSIdx_stat = 17;
#first angular frequency
GSIdx_low = 1;
NSIdx_low = 17;
#second angular frequency
GSIdx_med = 1;
NSIdx_med = 16;
#third angular frequency
GSIdx_high = 1;
NSIdx_high = 16;

Initialize the triangulation

In [8]:
triangulationOwn = readTriangulation(vertexFilename, elementFilename);

Load the static eigenmodes

In [9]:
ModeFile_stat = "Eigenvalues_"+domain+"_omega_0.00000.dat";
EVstat = readEigenmodes(ModeFile_stat)
GSstat = -EVstat[:,GSIdx_stat];
NSstat = EVstat[:,NSIdx_stat];

In [10]:
EVlow = readEigenmodes(ModeFile_low);
EVmed = readEigenmodes(ModeFile_med);
EVhigh = readEigenmodes(ModeFile_high);

Choose the appropriate dynamic eigenmodes

In [11]:
GS = np.zeros( (len(EVlow[:,0]), 3), dtype=complex );
GS[:,0] = EVlow[:,GSIdx_low]; 
GS[:,1] = EVmed[:,GSIdx_med];
GS[:,2] = EVhigh[:,GSIdx_high];

In [12]:
NS = np.zeros( (len(EVlow[:,0]), 3), dtype=complex );
NS[:,0] = EVlow[:,NSIdx_low]# - EVlow[:,17]; 
NS[:,1] = EVmed[:,NSIdx_med]#-EVmed[:,17];
NS[:,2] = EVhigh[:,NSIdx_high]#-EVhigh[:,17];

Compute the differences

In [13]:
#ninth state
NSdiff = np.zeros( (len(EVlow[:,0]), 3), dtype=complex );
NSdiff[:,0] = NS[:,0] - NSstat
NSdiff[:,1] = NS[:,1] - NSstat
NSdiff[:,2] = NS[:,2] - NSstat
#ground state
GSdiff = np.zeros( (len(EVlow[:,0]), 3), dtype=complex );
GSdiff[:,0] = GS[:,0] - GSstat
GSdiff[:,1] = GS[:,1] - GSstat
GSdiff[:,2] = GS[:,2] - GSstat

Define a function which will add interior labels to a figure

Draw the eigenmodes and their deviation from the static eigenmode for $\lambda_1,\lambda_9$ 
into two separate subplots.

An array plot combining all eigenmodes and their deviations is confusing.
Use 2 figures

Colorbar normalization for the eigenstates

In [14]:
vmaxGS = max(abs(GS[:,0:2].flatten()))
vminGS = np.real( min(GS[:,0:2].flatten()) )

vmaxNS = max(abs(NS[:,0:2].flatten()))
vminNS = np.real( min(NS[:,0:2].flatten()) )

normGS = matplotlib.colors.Normalize(vmax=vmaxGS, vmin=vminGS)
normNS = matplotlib.colors.Normalize(vmax=vmaxNS, vmin=vminNS)

Colorbar normalization for the differences

In [15]:
vmaxGSdiff = max(abs(GSdiff[:,0:2].flatten()))
vminGSdiff = np.real( min(GSdiff[:,0:2].flatten()) )

vmaxNSdiff = max(abs(NSdiff[:,0:2].flatten()))
vminNSdiff = np.real( min(NSdiff[:,0:2].flatten()) )

normGSdiff = matplotlib.colors.Normalize(vmax=vmaxGSdiff, vmin=vminGSdiff)
normNSdiff = matplotlib.colors.Normalize(vmax=vmaxNSdiff, vmin=vminNSdiff)

Draw the eigenmodes and their deviations for $\lambda_1$

In [16]:
PlotEigenmodes(GS, GSdiff, triangulationOwn, normGS, normGSdiff, xticklabels=[-3,-2,-1,0,1,2,3],\
               yticklabels=[-4,-3,-2,-1,0,1,2], show=True, save=False)

Do the same for $\lambda_9$

In [17]:
PlotEigenmodes(NS, NSdiff, triangulationOwn, normNS, normNSdiff, xticklabels=[-3,-2,-1,0,1,2,3],\
               yticklabels=[-4,-3,-2,-1,0,1,2], show=True, save=False)

Testbed for further plots

In [23]:
GSIdx_stat = 2;
NSIdx_stat = 16;
#first angular frequency
GSIdx_low = 0;
NSIdx_low = 16;
#second angular frequency
GSIdx_med = 0;
NSIdx_med = 18;
#third angular frequency
GSIdx_high = 3;
NSIdx_high = 16;

plt.figure()
plt.tripcolor(triangulationOwn,-np.real(EVstat[:,GSIdx_stat]), shading='gouraud', cmap=plt.cm.viridis)
plt.colorbar()
plt.tight_layout()
plt.show()

Determine the maximum of the ground state

In [24]:
GS = -np.real(EVstat[:,GSIdx_stat]);
maxIdx = np.argmax(GS)
print "x: ",triangulationOwn.x[maxIdx], " y: ", triangulationOwn.y[maxIdx]

x:  1.60417  y:  -0.453869
